Complete build in google colab

In [ ]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, MaxPool2D
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
IMAGE_SIZE = 224
labels = ['buffalo', 'elephant', 'monkey', 'tiger','wild boar']

In [ ]:
# DATA AUGMENTATION
# Balancing the no.of images to 300 in each class

import os
import shutil

# Count the number of images in each class
#class_counts = {}
#for class_name in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_name)
    if os.path.isdir(class_path):
        class_count = len(os.listdir(class_path))
        class_counts[class_name] = class_count

# Create a directory to store the augmented images
augmented_dir = os.path.join(base_dir, "augmented")
os.makedirs(augmented_dir, exist_ok=True)

# Augment the images to reach the desired number of images per class
for class_name, count in class_counts.items():
    if count < desired_images_per_class:
        class_dir = os.path.join(base_dir, class_name)
        augmented_class_dir = os.path.join(augmented_dir, class_name)
        os.makedirs(augmented_class_dir, exist_ok=True)

        # Calculate the number of images to generate
        num_augmented_images = desired_images_per_class - count

        # Generate augmented images
        for i in range(num_augmented_images):
            # Get a random image from the class directory
            image_file = os.listdir(class_dir)[i % count]
            image_path = os.path.join(class_dir, image_file)

            # Generate a new filename for the augmented image
            augmented_image_file = f"augmented_{i + 1}.jpg"
            augmented_image_path = os.path.join(augmented_class_dir, augmented_image_file)

            # Copy and augment the image
            shutil.copy(image_path, augmented_image_path)

In [ ]:
# Moving the augmented images to the original folders

#original_dataset_dir = '/content/drive/MyDrive/data1'
#augmented_dataset_dir = '/content/drive/MyDrive/data1/augmented'

#for class_folder in labels:
    augmented_class_folder = os.path.join(augmented_dataset_dir, class_folder)
    original_class_folder = os.path.join(original_dataset_dir, class_folder)

    # Move the augmented images to the original dataset class folder
    for filename in os.listdir(augmented_class_folder):
        source = os.path.join(augmented_class_folder, filename)
        destination = os.path.join(original_class_folder, filename)
        shutil.move(source, destination)

In [ ]:
def count_images(folder_path, file_extensions=[".jpg", ".jpeg", ".png"]):
    image_count = 0
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path) and os.path.splitext(file_name)[1].lower() in file_extensions:
            image_count += 1
    return image_count
# Example usage
folder_path = "/content/drive/MyDrive/S6/Mini Project/Ajmal/data1/wild boar"
num_images = count_images(folder_path)
print("Number of images:", num_images)

In [ ]:
def get_training_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        i = 1
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_arr = cv2.resize(img_arr, (IMAGE_SIZE, IMAGE_SIZE))
                data.append([resized_arr, class_num])
                if(i%20 == 0):
                  print(i,class_num)
                i+=1
            except Exception as e:
                print(e)
    print("Data loaded")
    print()
    return np.array(data, dtype=object)

dataset = get_training_data('/content/drive/MyDrive/Mini Project/data1')

In [ ]:
# Split the dataset into train, test, and validation sets
train_ratio = 0.7
test_ratio = 0.2
val_ratio = 0.1

# Split the dataset into train and remaining (test + validation)
train, remaining_data = train_test_split(dataset, train_size=train_ratio, random_state=42)

# Split the remaining data into test and validation
test, val = train_test_split(remaining_data, train_size=test_ratio/(test_ratio+val_ratio), random_state=42)

# Print the shapes of the resulting datasets
print("Train data shape:", train.shape)
print("Test data shape:", test.shape)
print("Validation data shape:", val.shape)

In [ ]:
# Split the data into features and labels

x_train = []
y_train = []
x_val = []
y_val = []
x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)
for feature, label in test:
    x_test.append(feature)
    y_test.append(label)
for feature, label in val:
    x_val.append(feature)
    y_val.append(label)
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))
print(len(x_val))
print(len(y_val))

In [ ]:
# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

In [ ]:
# Resize data for deep learning
x_train = x_train.reshape(-1, IMAGE_SIZE,IMAGE_SIZE, 3)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
y_test = np.array(y_test)

In [ ]:
# Convert class vectors to binary class matrices (one-hot encoding)
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=5)
y_val = to_categorical(y_val, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [ ]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

In [ ]:
for layer in resnet_model.layers:
    layer.trainable = False

In [ ]:
# Model 1

cnn=tf.keras.Sequential()
cnn.add(resnet_model)

cnn.add(tf.keras.layers.Conv2D(filters=64,padding='same',strides=2,kernel_size=3,activation='relu',input_shape=(IMAGE_SIZE,IMAGE_SIZE,3)))
cnn.add(BatchNormalization())
cnn.add(Conv2D(filters=32, padding='same', strides=2, kernel_size=3, activation='relu'))
cnn.add(BatchNormalization())
cnn.add(Conv2D(filters=32, padding='same', strides=2, kernel_size=3, activation='relu'))
cnn.add(BatchNormalization())

cnn.add(Flatten())
cnn.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
cnn.add(Dropout(0.5))
cnn.add(Dense(5, activation='softmax'))

cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Model 2

#model = tf.keras.Sequential()
model.add(resnet_model)

model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Model 3 without Resnet

#model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

In [ ]:
hist = cnn.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_val, y_val), callbacks=[reduce_lr, early_stop])

In [ ]:
print("Loss of the model is - ", cnn.evaluate(x_test, y_test)[0])
print("Accuracy of the model is - ", cnn.evaluate(x_test, y_test)[1] * 100, "%")

In [ ]:
cnn.save("/content/drive/MyDrive/model/Model 4, with Resnet early stopping(81 acc,loss 0.67).h5")

In [ ]:
# Specify the current location and the destination location of the file
source = '/content/Model P, without Resnet, Early Stop(85% acc).h5'
destination = '/content/drive/MyDrive/S6/Mini Project/user'

# Move the file to the new location
shutil.move(source, destination)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
predictions = cnn.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)  # Convert predictions to class labels
true_labels = np.argmax(y_test, axis=1)  # Convert true labels to class labels

In [ ]:
print(classification_report(true_labels, predicted_labels, target_names=labels))

In [ ]:
cm = confusion_matrix(true_labels, predicted_labels)
cm = pd.DataFrame(cm, index=labels, columns=labels)
plt.figure(figsize=(8,8))
sns.heatmap(cm, cmap="Blues", linecolor='black', linewidth=1, annot=True, fmt='', xticklabels=labels, yticklabels=labels)
plt.show()

In [ ]:
y_pred = cnn.predict(x_test)

fpr = dict()
tpr = dict()
roc_auc = dict()

num_classes = 5

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

plt.figure(figsize=(12,12))

# Plot ROC curves for each class
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for class %s' % (roc_auc[i], labels[i]))

# Plot micro-average ROC curve
plt.plot(fpr["micro"], tpr["micro"], label='Micro-average ROC curve (area = %0.2f)' % roc_auc["micro"], linestyle=':', linewidth=4)

# Add labels and legend
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

In [ ]:
correct = np.nonzero(predicted_labels == true_labels)[0]
incorrect = np.nonzero(predicted_labels != true_labels)[0]

In [ ]:
print("No. of correctly predicted images: ", len(correct))
print("No. of incorrectly predicted images: ", len(incorrect))

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(10, 8))
fig.suptitle('Correctly Classified Images', fontsize=16)
for i, idx in enumerate(correct[:6]):
    row = i // 3
    col = i % 3
    axes[row, col].imshow(x_test[idx])
    axes[row, col].set_title("Predicted Class: {}\nTrue Class: {}".format(labels[predicted_labels[idx]], labels[true_labels[idx]]))
    axes[row, col].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(10, 8))
fig.suptitle('Incorrectly Classified Images', fontsize=16)
for i, idx in enumerate(incorrect[:6]):
    row = i // 3
    col = i % 3
    axes[row, col].imshow(x_test[idx])
    axes[row, col].set_title("Predicted Class: {}\nTrue Class: {}".format(labels[predicted_labels[idx]], labels[true_labels[idx]]))
    axes[row, col].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
image = '/content/drive/MyDrive/Mini Project/data1/elephant/31c31736c6.jpg'

def preprocess_image(image_path):
    img_arr = cv2.imread(image_path)
    resized_arr = cv2.resize(img_arr, (IMAGE_SIZE, IMAGE_SIZE))
    preprocessed_img = np.array(resized_arr) / 255.0
    preprocessed_img = preprocessed_img.reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
    return preprocessed_img

preprocessed_img = preprocess_image(image)
prediction = cnn.predict(preprocessed_img)
predicted_class = labels[np.argmax(prediction)]
print("Predicted class:", predicted_class)